In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau # NEW IMPORT
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm
import numpy as np
import os

# --- 1. Dataset Class for Pre-extracted Features ---
class PreExtractedFeatureDataset(Dataset):
    """
    Fast dataset that loads pre-extracted features from memory/disk.
    """
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


# --- 2. Improved Classifier Model with Stronger Regularization ---
class FER_ImprovedClassifier(nn.Module):
    def __init__(self, feature_dim, num_classes=7):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            # INCREASED DROPOUT TO 0.5 for better regularization
            nn.Dropout(0.5), 
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            # INCREASED DROPOUT TO 0.5
            nn.Dropout(0.5), 
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.classifier(x)


# --- 3. Training Function with Learning Rate Scheduler ---
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, # SCHEDULER ADDED
                num_epochs, device='cpu', patience=10): # PATIENCE INCREASED
    """
    Training function incorporating a learning rate scheduler and early stopping.
    """
    model.to(device)
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        print(f"\n--- Epoch {epoch+1}/{num_epochs} --- (LR: {optimizer.param_groups[0]['lr']:.6f})")
        
        # Training phase
        model.train()
        train_correct = 0
        train_total = 0
        train_loss = 0.0
        
        train_bar = tqdm(train_loader, desc='Training')
        for features, labels in train_bar:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * features.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
            train_total += labels.size(0)
            
            train_bar.set_postfix({'Loss': f'{loss.item():.4f}', 'Acc': f'{train_correct/train_total:.4f}'})
        
        train_loss /= train_total
        train_acc = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_correct = 0
        val_total = 0
        val_loss = 0.0
        
        with torch.no_grad():
            for features, labels in tqdm(val_loader, desc='Validation'):
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * features.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
        
        val_loss /= val_total
        val_acc = val_correct / val_total
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")
        
        # SCHEDULER STEP: Adjust learning rate based on validation loss
        scheduler.step(val_loss)
        
        # Early stopping and saving
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), 'best_improved_model.pth')
            print(f"✓ New best model saved! (Val Acc: {val_acc*100:.2f}%)")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered after {epoch+1} epochs")
                break
    
    return model


# --- 4. Main Script ---
def main():
    FEATURES_FILE = "C:/adam/AMIT_Diploma/grad_project/extracted_features.pt"
    
    # Load pre-extracted features
    print("Loading pre-extracted features...")
    if not os.path.exists(FEATURES_FILE):
        print(f"Error: Features file not found at {FEATURES_FILE}. Please run the feature extraction step first.")
        return

    features_dict = torch.load(FEATURES_FILE)
    
    # Get feature dimension
    feature_dim = features_dict['train']['features'].shape[1]
    print(f"Feature dimension: {feature_dim}")
    
    # Prepare data split
    train_features = features_dict['train']['features']
    train_labels = features_dict['train']['labels']
    
    # Stratified split for train/validation
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
    train_idx, val_idx = next(splitter.split(
        np.arange(len(train_labels)),
        train_labels.numpy()
    ))
    
    # Create datasets
    train_dataset = PreExtractedFeatureDataset(
        train_features[train_idx],
        train_labels[train_idx]
    )
    val_dataset = PreExtractedFeatureDataset(
        train_features[val_idx],
        train_labels[val_idx]
    )
    test_dataset = PreExtractedFeatureDataset(
        features_dict['test']['features'],
        features_dict['test']['labels']
    )
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    
    print(f"Train samples: {len(train_dataset)}")
    print(f"Val samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")
    
    # Initialize model, loss, and optimizer
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Use the improved classifier
    model = FER_ImprovedClassifier(feature_dim=feature_dim, num_classes=7) 
    criterion = nn.CrossEntropyLoss()
    
    # ADDED WEIGHT DECAY (L2 Regularization)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) 
    
    # ADDED LEARNING RATE SCHEDULER
    scheduler = ReduceLROnPlateau(
        optimizer, 
        mode='min',      # Monitor minimum validation loss
        factor=0.5,      # Reduce LR by 50%
        patience=3,      # Wait 3 epochs before reducing LR
    )
    
    # Train
    print("\nStarting improved training...")
    trained_model = train_model(
        model, train_loader, val_loader, criterion, optimizer, scheduler, # Pass the scheduler
        num_epochs=50, device=device, patience=10 # Increased patience for scheduler
    )
    
    print("\nImproved training complete!")


if __name__ == "__main__":
    main()

Loading pre-extracted features...
Feature dimension: 1280
Train samples: 25838
Val samples: 2871
Test samples: 7178

Starting improved training...

--- Epoch 1/50 --- (LR: 0.001000)


Validation: 100%|██████████| 45/45 [00:00<00:00, 824.83it/s]


Train Loss: 1.4660, Train Acc: 44.22%
Val Loss: 1.2944, Val Acc: 50.68%
✓ New best model saved! (Val Acc: 50.68%)

--- Epoch 2/50 --- (LR: 0.001000)


Validation: 100%|██████████| 45/45 [00:00<00:00, 956.98it/s]


Train Loss: 1.3104, Train Acc: 50.57%
Val Loss: 1.2402, Val Acc: 53.05%
✓ New best model saved! (Val Acc: 53.05%)

--- Epoch 3/50 --- (LR: 0.001000)


...

Train Loss: 0.3822, Train Acc: 86.27%
Val Loss: 1.4241, Val Acc: 60.12%

--- Epoch 49/50 --- (LR: 0.000002)


Validation: 100%|██████████| 45/45 [00:00<00:00, 861.27it/s]


Train Loss: 0.3834, Train Acc: 86.42%
Val Loss: 1.4389, Val Acc: 59.91%

--- Epoch 50/50 --- (LR: 0.000002)


Validation: 100%|██████████| 45/45 [00:00<00:00, 768.00it/s]


Train Loss: 0.3834, Train Acc: 86.19%
Val Loss: 1.4242, Val Acc: 59.60%

Improved training complete!
